<a href="https://colab.research.google.com/github/sreramk1/sentiment-analysis/blob/main/training_notebooks/train_sentiment_p1v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training the Model for the first time (initialized with random weights)

In this notebook, I had used the classes I had written to simplify the training process for the sentiment analysis model: https://github.com/sreramk1/sentiment-analysis.git

In [1]:
!rm -r sentiment-analysis
!git clone https://github.com/sreramk1/sentiment-analysis.git

Cloning into 'sentiment-analysis'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 209 (delta 113), reused 140 (delta 58), pack-reused 0
Receiving objects: 100% (209/209), 8.83 MiB | 11.90 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [2]:
import os
import sys

sys.path.append('/content/sentiment-analysis')
os.environ['PYTHONPATH'] += ":/content/sentiment-analysis"

In [3]:
from util.jupyter_helpers import display_tf_ds

from train_validate_predict.pandas_read_ds_from_csv import PandasReadDatasetFromCSV
from sentiment_v1.preprocess_ds import TweetReviewCsvToPdDataFrame
from sentiment_v1.convert_ds_to_tensors import TweetReviewTfDataset
from sentiment_v1.model_builder import SentimentLSTMDense64Dense32Dense16Dense8Dense1
from train_validate_predict.model_train_validate_predict import ModelTrainValidatePredict
from model.keras_sequence_model_weights import KerasSequenceModelWeights

In [4]:
pd_dataset = PandasReadDatasetFromCSV("/content/sentiment-analysis/third_party/dataset/airline_sentiment_analysis.csv")
pd_dataset.load_dataset()
ds = pd_dataset.get_dataset()

In [5]:
tweet_review_csv_to_pd_dataframe = TweetReviewCsvToPdDataFrame(pd_dataset)
pd_train_ds = tweet_review_csv_to_pd_dataframe.prepare_and_get_train_ds()
pd_train_ds[0:10]

,Unnamed: 0,airline_sentiment,text
10959,13967,0,-- This is after you Cancelled Flightled our f...
6309,8383,1,you guys are lucky I love you
4926,6377,0,"I am scheduled for the morning, days after t..."
4287,5422,0,jumped the gun a little Cancelled Flighting ou...
2483,3101,0,days and not a word from Bad PR guys.
1534,1949,1,Very quick! TY. I am sorry to hear this. Can y...
1876,2358,1,Rhonda C. at Atlanta airport redeemed you guys...
10609,13561,0,not sure why we are being made to stand in lin...
8077,10534,0,"had baggage lost last night, no call from us a..."
8738,11322,0,...Loosing a lot of business by using Barclays...


In [6]:
pd_validate_ds = tweet_review_csv_to_pd_dataframe.prepare_and_get_validate_ds()
pd_validate_ds[0:10]

,Unnamed: 0,airline_sentiment,text
6841,9136,0,I will make sure to never use your incompetent...
3197,3998,1,Got me home amid snow &amp; Cancelled Flightla...
7266,9635,0,your a miserable airline and your loss of reve...
3530,4425,0,frustrating indeed especially when no one know...
1086,1420,0,"typical caned response from UA, fix your crapp..."
7605,10013,0,and no offers to provide us with a hotel or an...
11179,14231,0,- AA Cancelled Flighted on . - rebooked...
742,1016,0,Thank you. Mention that before I boarded she a...
3667,4614,1,"What can we do to bring you back to Jackson, M..."
6165,8188,0,why do you have every channel but How are and ...


In [7]:
tweet_review_tf_ds = TweetReviewTfDataset(tweet_review_csv_to_pd_dataframe)
tf_train_ds = tweet_review_tf_ds.prepare_and_get_train_ds()
display_tf_ds.display_tf_ds(tf_train_ds, 1)
tf_validate_ds = tweet_review_tf_ds.prepare_and_get_validate_ds()

(array([b"won't see my money again after their poor service    should have stuck with virgin america what was i thinking trying jetblue???",
       b'Jason (      ) at Gate in SAN made my afternoon!!!',
       b'Your MileagePlus signup page is broken.',
       b'That\xe2\x80\x99s a rather clunky process. Whoever designed that clearly doesn\xe2\x80\x99t understand e-mail    .',
       b'the B gates are very far though',
       b'yes. Bags came   hr after I arrived at baggage claim. Awful experience on the ground at BHM',
       b'your team member at DCA- Tamara R. is her name was awesome. You should have more employees like her!',
       b'have you read your own same day travel change policy?',
       b"when will airfare be available through December of this year? I have   major trips coming up and I'm anxious to book them!",
       b'Talk is cheap. Customer service is the action of making things right for the customer. again.',
       b"Awesome, thanks! I'll give a call Late Flightr to

In [8]:
model_builder = SentimentLSTMDense64Dense32Dense16Dense8Dense1(tf_ds_store=tweet_review_tf_ds)
model_builder.build_model()
model = model_builder.get_model()
model_trainer = ModelTrainValidatePredict(tweet_review_tf_ds, model)
model_trainer.predict("This is a bad airline")

array([[-0.00484287]], dtype=float32)

In [9]:
model_trainer.train()
model_trainer.evaluate()

Epoch 1/10
145/145 [==============================] - 21s 90ms/step - loss: 0.5771 - accuracy: 0.7920 - val_loss: 0.4096 - val_accuracy: 0.8081
Epoch 2/10
145/145 [==============================] - 10s 66ms/step - loss: 0.4017 - accuracy: 0.7920
Epoch 3/10
145/145 [==============================] - 10s 66ms/step - loss: 0.2846 - accuracy: 0.8233
Epoch 4/10
145/145 [==============================] - 9s 65ms/step - loss: 0.1759 - accuracy: 0.9411
Epoch 5/10
145/145 [==============================] - 10s 66ms/step - loss: 0.1222 - accuracy: 0.9606
Epoch 6/10
145/145 [==============================] - 10s 66ms/step - loss: 0.0920 - accuracy: 0.9717
Epoch 7/10
145/145 [==============================] - 9s 65ms/step - loss: 0.0689 - accuracy: 0.9810
Epoch 8/10
145/145 [==============================] - 10s 66ms/step - loss: 0.0544 - accuracy: 0.9859
Epoch 9/10
145/145 [==============================] - 10s 67ms/step - loss: 0.0408 - accuracy: 0.9897
Epoch 10/10
3/3 [=========================

(0.3877057135105133, 0.9120831489562988)

In [10]:
print("after training, bad review:" + str(model_trainer.predict("This is a bad airline")))
print("after training, good review:" + str(model_trainer.predict("This is a good airline")))

after training, bad review:[[-1.7543857]]
after training, good review:[[3.019926]]


In [11]:
sequence_model_weights = KerasSequenceModelWeights()
sequence_model_weights.set_model(model)
sequence_model_weights.read_weights_from_model()
sequence_model_weights.write_weights_to_file("weights.json")

In [12]:
another_sequence_model_weights = KerasSequenceModelWeights()
loaded_weights = another_sequence_model_weights.load_weights_from_file("weights.json")

second_model_builder = SentimentLSTMDense64Dense32Dense16Dense8Dense1(layered_weights=loaded_weights)
second_model_builder.build_model()
second_model = second_model_builder.get_model()
second_model_tvp = ModelTrainValidatePredict(tweet_review_tf_ds, second_model)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [13]:
another_sequence_model_weights.set_model(second_model)
another_sequence_model_weights.set_weights_to_model()

print("------------------------------------------------------------")
print("for a bad review" + str(second_model_tvp.predict("This is a bad airline")))
print("for a good review" + str(second_model_tvp.predict("This is a good airline")))
print("------------------------------------------------------------")

------------------------------------------------------------
for a bad review[[-1.7543857]]
for a good review[[3.019926]]
------------------------------------------------------------
